## Demonstration of the Reflection Pattern for Agentic Systems - general chatbot/assistant

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate

### Three prompts for this pattern:
- Draft
- Critique
- Revise

In [4]:
DRAFT_PROMPT = """
You are a helpful assistant that can answer questions and help with tasks. Produce
a first draft of an answer to the user's question.
"""

draft = ChatPromptTemplate.from_messages([
    ("system", DRAFT_PROMPT),
    ("human", "{task}")
])

In [5]:
CRITIQUE_PROMPT= """
You are a strict reviewer. Evaluate the draft response given the task.

Return exactly:
VERDICT: Pass or Fail

IF FAIL then add:
ISSUES:
- <bullet>
- <bullet>
"""

critique = ChatPromptTemplate.from_messages([
    ("system", CRITIQUE_PROMPT),
    ("human", "TASK: {task}  DRAFT: {draft}")
])

In [6]:
REVISE_PROMPT = """
You are skilled at revising drafts given reviewer feedback.

Your instructions, given a task, the first draft and the feedback on the draft do the following:
- Fix every issue
- Be concise
- Output ONLY the revised answer
"""

revise = ChatPromptTemplate.from_messages([
    ("system", REVISE_PROMPT),
    ("human",  "TASK: {task} DRAFT: {draft} REVIEW:{review} Write the revised answer:")
])

### Create the LangChain Runnables / Chains
#### More simply - feeding the prompt to the llm

In [7]:
# '|' operator creates a RunnableSequence
draft_chain = draft | llm
critique_chain = critique | llm
revise_chain = revise | llm

### Reflection Loop Function

In [8]:
def reflection(task, max_loops):
    # invoke - call the chain
    draft = draft_chain.invoke({"task": task}).text
    print("First Draft: ", draft)
    
    for i in range(max_loops):
        review = critique_chain.invoke({"task": task, "draft": draft}).text
        
        print(f"Review {i+1}:\n", review)
        
        if "verdict: pass" in review.lower():
            return draft
        
        draft = revise_chain.invoke({"task":task, "draft":draft, "review":review}).text
        print(f"Revision {i+1}:\n", draft)
        
    return draft        
        

### Testing the Loop

In [9]:
task = """I want to learn about the correlation between linear algebra and Artificial Intelligence. 

Constraints:
- 6 sentences max
- Assume I understand the basics of linear algebra
- Dont get too technical or equation heavy
"""

final_answer = reflection(task, max_loops=3)
print("Final Answer:\n", final_answer)

First Draft:  Linear algebra is the fundamental mathematical language underpinning almost all of Artificial Intelligence. At its core, AI processes information, and this information—whether images, text, or numerical data—is represented as vectors, matrices, and tensors. Operations like data transformation, feature extraction, and model predictions are essentially complex sequences of linear algebraic computations. For instance, in machine learning, algorithms often involve matrix multiplications to weigh features or solve systems of equations. Deep learning, a prominent AI subfield, heavily relies on linear algebra for neural network architectures, backpropagation, and optimizing model parameters. Therefore, a solid grasp of linear algebra is crucial for understanding how AI models work, designing new algorithms, and interpreting their behavior.
Review 1:
 VERDICT: Pass
Final Answer:
 Linear algebra is the fundamental mathematical language underpinning almost all of Artificial Intelli